In [ ]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# set figure size
plt.rcParams["figure.figsize"] = (8,6)

# specify decimal places to display
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Datetime objects in Python, Numpy and Pandas

##### Native Python dates and times

In [ ]:
# Python: datetime module

from datetime import datetime
d = datetime(year=2021, month=6, day=30)
d

In [ ]:
# Once you have a datetime object, you can do things like printing the day of the week:

d.strftime('%B')

#### Numpy native time series dtype

In [ ]:
date = np.array('2015-07-14', dtype=np.datetime64)
date

In [ ]:
# Now can quickly do vectorized operations on it!

date + np.arange(1,5)

In [ ]:
# https://numpy.org/doc/stable/reference/arrays.datetime.html
# np.datetime64('2015-07-04 12:59:59')

##### Pandas time series data type:

- Timestamp
- 'DatetimeIndex' - Datetime object as Index

In [ ]:
t = pd.Timestamp('2021-06-30, 07:50:15')
t

In [ ]:
# Can pandas read these dates?

# dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
#                        '2015-Jul-6', '07-07-2015', '20150708'])
# dates

In [ ]:
# We can use various attributes of datetime object to figure out day, month, year, weekday etc
t.quarter

In [ ]:
# weekday?

t.weekday()
t.day_name()

Timestamp documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html

## Demo: Working with Timeseries data in pandas

In [ ]:
# We will use Ethererum price data from 2017-2020 to demo Pandas capability for time-series data

sheet_url = 'https://docs.google.com/spreadsheets/d/1kTrAFSrr-xP3REs0Lly0TdV4ekrHahBXLg9r5qKxmV8/edit#gid=0'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

df = pd.read_csv(csv_export_url)
df.head(10)

In [ ]:
df.info()

In [ ]:
# check for nulls
df.isnull().sum()

In [ ]:
# plt.plot(df.Date, df.Close )

In [ ]:
# use pandas .plot()

df.Close.plot()

### Time series analysis: How to make pandas time-aware and leverage inbuild functionality:

1. Convert 'date/time' column to datetime object in Pandas (pd.to_datetime())
2. Set the datetime column as Index
3. Sort the datetime index

In [ ]:
# pandas to_datetime method can convert strings/object to datetime object (aka Timestamp object).
# Most of time pandas is smart enough to parse the date without any help.

pd.to_datetime('Jan 1 1970')

In [ ]:
# Can pandas read these dates?

# dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
#                        '2015-Jul-6', '07-07-2015', '20150708'])
# dates

In [ ]:
# Can pandas read this datetime string?
# pd.to_datetime('2020-03-13 08-PM')

In [ ]:
# Sometimes we have to specify the format of the datatime string to help pandas parse the date/time)

pd.to_datetime('2020-03-13 08-PM', format='%Y-%m-%d %I-%p')

Documentation on strftime() Format Codes:  
https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
df.head()

In [ ]:
# Step 1: Lets convert the 'Date' column in our df to pandas datetime object using pd.to_datetime()

df.Date = pd.to_datetime(df.Date, format = '%Y-%m-%d %I-%p')

In [ ]:
# confirm if the 'Date' column is indeed a datetime object now

df.Date

In [ ]:
# Step 2 and 3 combined: Set 'Date' column as Index and sort the index:

df = df.set_index('Date').sort_index()

In [ ]:
df.head(5)

In [ ]:
# What is the Index type now?

type(df.index)

In [ ]:
# lets plot the closing price again and how it compares to previous plot

df.Close.plot()

#### Pandas *datetimeIndex* attributes

https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html

In [ ]:
df.head()

In [ ]:
# add a new columns called 'day' which is the 'weekday'

df['day'] = df.index.day_name()

In [ ]:
df.head()

In [ ]:
# Can I use groupby to see mean trading volume by day?

### How to choose subset of a dataframe?

In [ ]:
df.head()

In [ ]:
df.loc['2017-07-01 11:00:00']

In [ ]:
# Turn the argument into something pandas interprets as a range
df['2017': '2018']

In [ ]:
# choose rows for '2017-07-01' (show only first 10 values)

df['2017-07-01'][:10]

In [ ]:
# we can also use date range like this

df['2018-02-01':'2018-02-02'].head(10)

### Changing the Period

- Upsampling - increasing frequency
    - This will result in empty observations which you may choose to fill with .ffill(),  .bfill(), or some other method.

- Downsampling - reducing frequency

In [ ]:
df.head()

In [ ]:
# Upsample using asfreq() method

# How can we take care of new NaNs introduced in our data by asfreq method?

df.asfreq('45Min', method = 'ffill')

#### What did we do our dataframe (size?)



#### Code Description   	
D	Calendar day	       
W	Weekly		  
M	Month end	          
Q	Quarter end            
A	Year end	         
H	Hours	              
T	Minutes		
S	Seconds		
L	Milliseonds		
U	Microseconds		
N	nanoseconds		
   


In [ ]:
#  how can we backfill the data:

df.asfreq('45Min', method = 'bfill')

In [ ]:
#Upsample, What if I don't want to use bfill or ffill

df.asfreq('45Min')

In [ ]:
# You can also do use fillna to fill NaNs with certain chosen value:

df.asfreq('45min').fillna(0)

When to use 'ffill' or 'bfill' or None?

What is another option to fill NaNs, rather than 'ffill', 'bfill' or None?


#### How can we go from more frequent to less frequent (Downsample)

In [ ]:
# We can use .asfreq method. Here we go from data every hour to every 3 hours

#downsample

# show only first 10 values:
df.asfreq('3H')[:10]

In [ ]:
# selecting data based on daily frequency:

df.asfreq('D')[:10]

#### What does the code above do to your dataframe content and size?

### Resampling 

- Fundamentally data aggregation (similar to groupby but for dates)
    - as opposed to afreq which is more of a 'data selection'
- Only works on dataframes with a datetime index


In [ ]:
df.head()

In [ ]:
# resample the whole dataframe every hour using mean as an aggregator

df.resample('D').mean()

In [ ]:
df.head()

In [ ]:
# resample monthly and use mean for aggregation

df.resample('M').mean().head()

In [ ]:
# resample yearly
df.resample('Y').mean().head()

In [ ]:
# Plot raw data with monthly resampled data

df.Close.plot(label = 'raw data')
df.resample('M').mean().Close.plot(label = 'Monthly Resampled')
plt.legend()

### Rolling Averages/Windows
Why use rolling average
- Rolling average aka. moving averages, are generally used to smooth out short-term fluctuations in time series data and highlight long-term trends

In [ ]:
df.head()

In [ ]:
# What is the rolling average of 4 hours?

df.rolling(4).mean()

What if I want to generate a rolling window function of weekly data from hourly data?

In [ ]:
# Resampling and rolling average together

df.resample('W').mean().Close.rolling(4).mean()

In [ ]:
# Resampling and rolling average together (26 weeks rolling average)

df.resample('W').mean().Close.rolling(26).mean()

In [ ]:
# %matplotlib notebook


df.Close.plot(label = 'raw data')
df.resample('W').mean().Close.rolling(2).mean().plot(label = '2W rolling')
df.resample('W').mean().Close.rolling(8).mean().plot(label = '8W rolling')
df.resample('W').mean().Close.rolling(26).mean().plot(label = '26W rolling')
plt.legend()

#### What other rolling averages can we use?


### Lagging and Lead

- Shift: move the data backwards and forwards by a given amount
- diff: find the difference with the previous observation (or a specified further back observation)
- Can be used to computing differences over time

In [ ]:
df_daily = df.resample('D').mean()
df_daily.head(10)

In [ ]:
# Use 'shift' to move the timeseries into the future

df_daily['shift(1)'] = df_daily.Close.shift(1)
df_daily.head(10)

In [ ]:
# create a new column 'diff(1)'
df_daily['diff(1)'] = df_daily.Close.diff(1)
df_daily.head(10)

In [ ]:
plt.plot(df.Close)

In [ ]:
# %matplotlib inline
# Use for shift/diff to calculate 1 year rate of return

((df_daily.Close.shift(-365)/df_daily.Close -1) *100).plot()
plt.ylabel('% Return on Investment')
plt.title('1-Year Rate of return on ETH')
plt.xlim(pd.Timestamp('2017-07'), pd.Timestamp('2019-06'))

### Time Zone

In [ ]:
# datetime with no timezone specified (sometimes called timezone naive datetime object)

t = pd.Timestamp('2020-04-14 10:20:20')
t

In [ ]:
#create a datetime object with specify timezone

# t1 = pd.Timestamp('2020-04-14, 07:50', tz = 'US/Central')
# t1

In [ ]:
# localize a naive timezone
t_localized = t.tz_localize('US/Central')
t_localized

In [ ]:
# convert to a different timezone

t_localized.tz_convert(tz = 'Asia/tokyo')

In [ ]:
# Can I convert a naive timestamp to a particular timezone?


### Timedelta

In [ ]:
# What is date and time right now? Take any datetime object and use .now() method

pd.Timestamp.now()

In [ ]:
# time delta between now (today's date) and t

(pd.Timestamp.now() - t)

In [ ]:
# Can we find time delta between t and t1?

t, t_localized

In [ ]:
# To convert the above Timedelta object to a number:

(pd.Timestamp.now() - t) // pd.Timedelta('1d')

### Bonus

In [ ]:
# Bonus: How to remove timezone info from a datetime object

t1 = t1.tz_localize(None)
t1

In [ ]:
# Bonus: What is going on here?

pd.Timestamp('2021-03-14 02:00', tz = 'US/Eastern')